In [1]:
import os
import pandas as pd

In [5]:
def combine_csvs_from_subdirectories(root_directory):
    """
    Reads all CSV files from the subdirectories of the root directory and combines them into a single DataFrame.

    Args:
        root_directory (str): Path to the root directory containing subdirectories with CSV files.

    Returns:
        pd.DataFrame: Combined DataFrame containing data from all CSV files.
    """
    all_csv_files = []

    # Walk through the root directory and its subdirectories
    for subdir, _, files in os.walk(root_directory):
        files = [x for x in files if x.endswith(".csv")]
        for file in files:
            full_path = os.path.join(subdir, file)
            all_csv_files.append(full_path)
    all_csv_files = [
        x for x in all_csv_files if "checkpoint" not in x and "combined" not in x
    ]

    combined_df = pd.DataFrame()
    for path in all_csv_files:
        print(path)
        temp = pd.read_csv(path, on_bad_lines="skip", dtype="unicode")

        print(f"subsetting to max 100 instances of each order, going from {temp.shape[0]} rows to....")
        temp = temp.groupby('order_name').apply(lambda x: x.sample(n=min(len(x), 100), replace=False)).reset_index(drop=True)

        print(f"...{temp.shape[0]}")
        combined_df = pd.concat([combined_df, temp], ignore_index=True)

    return combined_df

In [ ]:
root_dir = "/home/users/katriona/amber-inferences/data/harlequin_new_loc_model/"
combined_dataframe = combine_csvs_from_subdirectories(root_dir)

In [ ]:
combined_dataframe